In [1]:
from geopy.distance import geodesic
import osmium
import os
import math
from scipy.spatial import KDTree
import numpy as np
import random

out_file = "../osm/cost.osm"
input_file = '../osm/divided.osm'

In [2]:
def check_cost(length, lanes):
    return length / math.sqrt(lanes)

In [3]:
class OSMHandler(osmium.SimpleHandler):
    def __init__(self):
        super().__init__()
        self.node_locs = osmium.index.create_map("sparse_mem_array")
        self.loc_handler = osmium.NodeLocationsForWays(self.node_locs)
        self.ways = []
        self.builsings = []
        self.nods_way = []
        self.nods_build = []
        
    def node(self, n: osmium.Node):
        self.node_locs.set(n.id, n.location)

    def way(self, w: osmium.Way):
       
        if 'highway' in w.tags:
            total_length = 0.0
            lanes = float(w.tags.get('lanes', '2'))
            prev_node = None
            for n in w.nodes:
                if n.location.valid():
                    if prev_node is not None:
                        total_length += geodesic((prev_node.lat, prev_node.lon), (n.location.lat, n.location.lon)).meters
                    prev_node = n.location 
            cost = check_cost(total_length, lanes)
            new_way = osmium.osm.mutable.Way(w)
            new_way.tags = dict(w.tags)
            new_way.tags['cost'] = f"{cost:.2f}"
            self.ways.append((w.id, dict(w.tags), cost, list(w.nodes)))


In [4]:
def save_modified_ways(ways, nodes_way, writer):

    for id, loc in nodes_way:
        node = osmium.osm.mutable.Node()
        node.id = id
        node.location = loc 
        node.tags = {}
        writer.add_node(node)
        
    for way_id, tags, cost, nods in ways:
        new_way = osmium.osm.mutable.Way()
        new_way.id = way_id
        new_way.tags = tags
        new_way.tags['cost'] = f"{cost:.2f}"
        new_way.nodes = nods
        writer.add_way(new_way)
    writer.close()

In [6]:

if os.path.exists(out_file):
    os.remove(out_file)

writer = osmium.SimpleWriter(out_file)

try:
    h = OSMHandler()
    h.apply_file(input_file, locations=True)
    save_modified_ways(h.ways, h.nods_way, writer)
finally:
    writer.close()  